Remove useless neurons

In [8]:
import pandas as pd

mean_iou = {}
i = 5
res = pd.read_csv(f"result_{i}.csv")
res = res[res["iou"] > 0]
res = res.sort_values(by="iou", ascending=False)
mean_iou[i] = res["iou"].mean()
res.to_csv(f"result_{i}_parsed.csv", index=False)

find 4 outputs top connections

In [9]:
import pandas as pd

i = 5
N = 10
csv = pd.read_csv(f"result_{i}_parsed.csv")
output_file = f"output_{i}.txt"
neurons = []


def find_unique_elements(arr1, arr2, arr3, arr4):
    set1, set2, set3, set4 = set(arr1), set(arr2), set(arr3), set(arr4)
    unique1 = set1 - set2 - set3 - set4
    unique2 = set2 - set1 - set3 - set4
    unique3 = set3 - set1 - set2 - set4
    unique4 = set4 - set1 - set2 - set3

    return unique1, unique2, unique3, unique4


for label in ["w_nothing", "w_left", "w_main", "w_right"]:
    sorted_csv = csv.sort_values(by=label, ascending=False)
    top_neurons = sorted_csv.head(N)["neuron"].tolist()
    neurons.append(top_neurons)

unique_elements = find_unique_elements(neurons[0], neurons[1], neurons[2], neurons[3])
print(unique_elements, "\n")

with open(output_file, "w") as f:
    for i, label in enumerate(["w_nothing", "w_left", "w_main", "w_right"]):
        csv = csv.sort_values(by=label, ascending=False)
        top_neurons = csv.head(N)[["neuron", label]].to_records(index=False)
        print(f"\n============\n{label} top {N}:", file=f)
        for neuron, value in top_neurons:
            if neuron in unique_elements[i]:
                print(f"* {neuron} {value}", file=f)  # Special mark for unique elements
                print(f"* {neuron} {value}")
            else:
                print(f"  {neuron} {value}", file=f)
                print(f"  {neuron} {value}")

({8, 58, 4, 36}, set(), {54}, set()) 

  43 3.0032465
  50 2.4275603
  42 2.1019106
  29 2.0634496
  11 1.7551857
* 58 0.9783437
* 4 0.9145632
  6 0.5453575
* 8 0.32049042
* 36 0.1964968
  5 2.2308052
  41 2.0272539
  29 2.0034888
  6 1.6569151
  43 1.6063035
  1 1.2202142
  0 0.7341674
  11 0.6483451
  42 0.52189475
  19 0.4329153
  50 2.1955557
  29 1.8131213
  42 1.7148129
  0 1.5977954
  5 1.4379873
  41 1.3720415
* 54 1.096387
  43 1.0882556
  11 0.96671075
  1 0.9040145
  29 2.427602
  5 1.6537093
  11 1.5697021
  41 1.3866223
  43 1.3612376
  19 1.2705342
  42 1.2181396
  50 1.1292526
  6 0.5535836
  0 0.5304088


In [14]:
class LogicalExpressionParser:
    def __init__(self, operators):
        self.operators = operators
        self.tokens = []
        self.current = 0

    def tokenize(self, expression):
        """将表达式字符串转换为token列表"""
        # 替换括号为带空格的形式，便于分割
        expression = expression.replace("(", " ( ").replace(")", " ) ")
        # 分割并移除空字符串
        self.tokens = [token for token in expression.split() if token]
        self.current = 0

    def peek(self):
        """查看当前token"""
        if self.current < len(self.tokens):
            return self.tokens[self.current]
        return None

    def advance(self):
        """移动到下一个token"""
        self.current += 1
        return self.tokens[self.current - 1]

    def parse(self, expression):
        """解析表达式"""
        self.tokenize(expression)
        return self.parse_expression()

    def parse_expression(self):
        """解析表达式"""
        if self.peek() == "(":
            return self.parse_group()
        elif self.peek() == "NOT":
            self.advance()  # consume NOT
            operand = self.parse_expression()
            return lambda x: ~operand(x)
        else:
            # 原子概念
            token = self.advance()
            return self.operators[token]

    def parse_group(self):
        """解析括号组"""
        self.advance()  # consume (
        left = self.parse_expression()

        while self.peek() in ["AND", "OR"]:
            operator = self.advance()
            right = self.parse_expression()
            
            if operator == "AND":
                left = self.create_and(left, right)
            else:  # OR
                left = self.create_or(left, right)

        self.advance()  # consume )
        return left

    @staticmethod
    def create_and(f1, f2):
        return lambda x: f1(x) & f2(x)

    @staticmethod
    def create_or(f1, f2):
        return lambda x: f1(x) | f2(x)

def parse_logical_expression(expression, operators):
    parser = LogicalExpressionParser(operators)
    return parser.parse(expression)

lunarv3_operators = {
    "X1": lambda inp: (inp[:, 0] > -0.25) & (inp[:, 0] < 0),
    "X2": lambda inp: (inp[:, 0] >= 0) & (inp[:, 0] < 0.25),
    "X3": lambda inp: (inp[:, 0] > -0.4) & (inp[:, 0] <= -0.25),
    "X4": lambda inp: (inp[:, 0] < 0.4) & (inp[:, 0] >= 0.25),
    "X5": lambda inp: (inp[:, 0] <= -0.4),
    "X6": lambda inp: (inp[:, 0] >= 0.4),
    "Y1": lambda inp: (inp[:, 1] <= 0.1),
    "Y2": lambda inp: (inp[:, 1] > 0.1) & (inp[:, 1] <= 0.2),
    "Y3": lambda inp: (inp[:, 1] > 0.2) & (inp[:, 1] <= 0.3),
    "Y4": lambda inp: (inp[:, 1] > 0.3) & (inp[:, 1] <= 0.4),
    "Y5": lambda inp: (inp[:, 1] > 0.4) & (inp[:, 1] <= 0.5),
    "Y6": lambda inp: (inp[:, 1] > 0.5) & (inp[:, 1] <= 0.7),
    "Y7": lambda inp: (inp[:, 1] > 0.7),
    "Vx1": lambda inp: (inp[:, 2] >= -0.1) & (inp[:, 2] < 0),  # Vx low
    "Vx2": lambda inp: (inp[:, 2] >= 0) & (inp[:, 2] <= 0.1),  # Vx low
    "Vx3": lambda inp: (inp[:, 2] > 0.1) & (inp[:, 2] <= 0.2),  # Vx slightly positive
    "Vx4": lambda inp: (inp[:, 2] > 0.2) & (inp[:, 2] <= 0.4),  # Vx moderately positive
    "Vx5": lambda inp: (inp[:, 2] > 0.4) & (inp[:, 2] <= 1.0),  # Vx high positive
    "Vx6": lambda inp: (inp[:, 2] < -0.1) & (inp[:, 2] >= -0.2),  # Vx slightly negative
    "Vx7": lambda inp: (inp[:, 2] < -0.2)
    & (inp[:, 2] >= -0.4),  # Vx moderately negative
    "Vx8": lambda inp: (inp[:, 2] < -0.4) & (inp[:, 2] >= -1.0),
    "Vy1": lambda inp: (inp[:, 3] >= -0.1) & (inp[:, 3] <= 0.0),  # Vy low downward
    "Vy2": lambda inp: (inp[:, 3] < -0.1) & (inp[:, 3] >= -0.2),  # Vy slightly downward
    "Vy3": lambda inp: (inp[:, 3] < -0.2)
    & (inp[:, 3] >= -0.4),  # Vy moderately downward
    "Vy4": lambda inp: (inp[:, 3] < -0.4) & (inp[:, 3] >= -1.0),  # Vy high downward
    "Vy5": lambda inp: (inp[:, 3] > 0.0) & (inp[:, 3] <= 0.2),  # Vy slightly upward
    "Vy6": lambda inp: (inp[:, 3] > 0.2) & (inp[:, 3] <= 0.4),  # Vy moderately upward
    "Vy7": lambda inp: (inp[:, 3] > 0.4) & (inp[:, 3] <= 1.0),
    "A1": lambda inp: (inp[:, 4] <= -1.0),  # Angle very large (left)
    "A2": lambda inp: (inp[:, 4] > -1.0)
    & (inp[:, 4] <= -0.15),  # Angle moderately large (left)
    "A3": lambda inp: (inp[:, 4] >= -0.15) & (inp[:, 4] <= 0),  # Angle near 0 (center)
    "A4": lambda inp: (inp[:, 4] >= 0) & (inp[:, 4] <= 0.15),  # Angle near 0 (center)
    "A5": lambda inp: (inp[:, 4] > 0.15)
    & (inp[:, 4] <= 1.0),  # Angle moderately large (right)
    "A6": lambda inp: (inp[:, 4] > 1.0),
    "AV1": lambda inp: (inp[:, 5] <= -0.25),  # Angular velocity very high (left)
    "AV2": lambda inp: (inp[:, 5] > -0.25)
    & (inp[:, 5] <= -0.1),  # Angular velocity moderately high (left)
    "AV3": lambda inp: (inp[:, 5] >= -0.1)
    & (inp[:, 5] <= 0),  # Angular velocity low (center)
    "AV4": lambda inp: (inp[:, 5] >= 0)
    & (inp[:, 5] <= 0.1),  # Angular velocity low (center)
    "AV5": lambda inp: (inp[:, 5] > 0.1)
    & (inp[:, 5] <= 0.25),  # Angular velocity moderately high (right)
    "AV6": lambda inp: (inp[:, 5] > 0.25),
    "LLeg": lambda inp: inp[:, 6] == 1,
    "RLeg": lambda inp: inp[:, 7] == 1,
}

# 测试表达式
test_expression = "(LLeg OR RLeg)"

# 解析表达式
combined_function = parse_logical_expression(test_expression, lunarv3_operators)

# 测试结果
import numpy as np
# 创建测试数据
states = np.load("states.npy")


# 使用组合函数
result = combined_function(states)
# 获取 True 的索引
indices = np.where(result)[0][:10]
print("Result:", states[indices])

Result: [[ 1.8729210e-01  6.0042236e-03 -4.5699798e-02 -2.1145369e-01
   7.7426016e-02 -1.8042313e-02  0.0000000e+00  1.0000000e+00]
 [ 1.8670006e-01  2.3698306e-03 -4.0064070e-02 -1.6092648e-01
   5.7646915e-02 -3.9668244e-01  0.0000000e+00  1.0000000e+00]
 [ 1.8610191e-01 -5.7966949e-04 -2.3902554e-02 -1.3010798e-01
   2.2112370e-02 -7.1076018e-01  0.0000000e+00  1.0000000e+00]
 [ 1.8539505e-01 -2.3948455e-03 -2.7449271e-02 -8.0603443e-02
  -2.0634277e-02 -8.5502023e-01  0.0000000e+00  1.0000000e+00]
 [ 1.8455735e-01 -3.9822124e-03 -4.1613456e-02 -7.1656279e-02
  -6.2279489e-02 -8.3290499e-01  0.0000000e+00  1.0000000e+00]
 [ 1.8101224e-01 -1.1202540e-02 -6.4120926e-02 -1.2287087e-01
  -2.1799175e-01 -7.1984541e-01  1.0000000e+00  0.0000000e+00]
 [ 1.8058309e-01 -1.2690344e-02 -2.4191488e-02 -6.9224559e-02
  -2.3536010e-01 -3.4526211e-01  1.0000000e+00  0.0000000e+00]
 [ 1.8080597e-01 -1.3103797e-02  1.9030128e-02 -1.7857213e-02
  -2.3203172e-01  6.6588745e-02  1.0000000e+00  0.00000

find act & inp pair

In [2]:
import numpy as np

act = np.load(f"../save/LunarLander-{NET_NAME}/actions.npy")
obs = np.load("../save/lunar.npy")

act_list = []
act_idx_list = []
max_indices = np.argmax(act, axis=1)
print(max_indices)
print(max_indices.shape)
for i in range(4):
    max_row_index = np.where(max_indices == i)[0]
    print(max_row_index.shape)
    # max_row = act[max_row_index]
    act_idx_list.append(max_row_index)
    act_list.append(obs[max_row_index])

# np.save(f'../save/LunarLander-{NET_NAME}/max_indices.npy', max_indices)

[3 3 3 ... 1 1 1]
(10000,)
(1715,)
(2127,)
(2696,)
(3462,)


## analyze specific neuron(DQN)

In [26]:
from analyze import *

ckpt_path = "../save/ckpt-mlp1024-260.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
model, dataset = load_for_analysis(ckpt_path, data_path)
weights = model.fc3.weight.t().detach().cpu().numpy()

print("Extract features")
inputs, features, outputs = extract_feature(model, dataset)
outputs = np.array(outputs)
if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Extract features


100%|██████████| 157/157 [00:00<00:00, 419.39it/s]


Computing quantiles
Generating atomic masks
(10000, 16)


[x, y, Vx, Vy, angle, angular_v, l, r]

y_near_ground: <0.4

Vy_low: -0.5 ~ 0

In [39]:
def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


vy_low = np.where(feat_masks[:, 7] > 0)[0]
y_near_ground = np.where(feat_masks[:, 4] > 0)[0]

vy_low_and_y_near_ground = intersect(vy_low, y_near_ground)

neuron_act = np.where(activations[:, 630] > 0)[0]
res = intersect(neuron_act, act_idx_list[0])
res = intersect(res, vy_low_and_y_near_ground)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][630])
hook = HookLayer()
hook.hook_layer(layer=model.act2)
obs_attk = obs[res[10]]
obs_attk[2] = 1.0
print(obs_attk)
act_attk = model(torch.tensor(obs_attk, dtype=torch.float32).to(settings.DEVICE))
print(hook.features_blobs)

(1237,)
[54.300285 52.586964 50.26787  52.025227]
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
0.63623697
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]


In [40]:
print(hook.features_blobs[0][630])
print(act_attk)

0.0
tensor([88.7608, 86.5204, 93.6968, 86.7465], device='cuda:3',
       grad_fn=<ViewBackward0>)


## analyze attack (PPO)


In [3]:
from PPO import get_PPO
import settings
from analyze import *

ckpt_path = "../save/LunarLander-PPO1024/ppo.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
inputs, features, outputs, weight, policy = get_PPO(ckpt_path, data_path)


if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Computing quantiles
Generating atomic masks
(10000, 16)


In [18]:
import tianshou


def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


not_l_leg = np.where(feat_masks[:, 14] == 0)[0]
r_leg = np.where(feat_masks[:, 15] > 0)[0]
x_in_centor = np.where(feat_masks[:, 0] > 0)[0]

inter = intersect(not_l_leg, r_leg)
inter = intersect(x_in_centor, inter)

neuron_act = np.where(activations[:, 212] > 0)[0]
res = intersect(neuron_act, act_idx_list[2])
res = intersect(res, inter)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][212])
hook = HookLayer()
hook.hook_layer(policy.actor.preprocess.model.model[2])
obs_attk = obs[res[10]]
obs_attk[0] = 0.9
obs_attk[6] = 1.0
obs_attk = obs_attk.reshape(1, -1)
print(obs_attk)
print(obs_attk.shape)
act_attk = policy(tianshou.data.Batch(obs=obs_attk, info=""))
print(hook.features_blobs[0][0][212])
print(act_attk)

(14,)
[0.28012165 0.15548752 0.48146957 0.0829213 ]
[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
  1.          1.        ]
0.37163532
[[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
   1.          1.        ]]
(1, 8)
-1.70284
Batch(
    logits: tensor([[1.2064e-01, 8.7784e-01, 1.1213e-03, 3.9262e-04]], device='cuda:3',
                   grad_fn=<SoftmaxBackward0>),
    act: tensor([1], device='cuda:3'),
    state: None,
    dist: Categorical(probs: torch.Size([1, 4])),
)
